In [1]:
%pip install carla 
%pip install opencv-python
%pip install pygame

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import the CARLA Python API library 
import carla
import math
import random
import time
import queue
import numpy as np
import cv2
import pygame
from PIL import Image
import json

pygame 2.6.0 (SDL 2.28.4, Python 3.8.19)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
# Connect to the client and get the world object
client = carla.Client('localhost', 2000) 
client.set_timeout(200.0)
world  = client.reload_world()
bp_lib = world.get_blueprint_library()
spawn_points = world.get_map().get_spawn_points() 

In [4]:
#small town with rain and river
#client.load_world('Town01')


In [5]:
#change weather to rainy
weather = carla.WeatherParameters(
    cloudiness=30.0,
    precipitation=70.0,
    sun_altitude_angle=70.0)

world.set_weather(weather)

print(world.get_weather())

WeatherParameters(cloudiness=30.000000, precipitation=70.000000, precipitation_deposits=0.000000, wind_intensity=0.000000, sun_azimuth_angle=0.000000, sun_altitude_angle=70.000000, fog_density=0.000000, fog_distance=0.000000, fog_falloff=0.000000, wetness=0.000000, scattering_intensity=0.000000, mie_scattering_scale=0.000000, rayleigh_scattering_scale=0.033100, dust_storm=0.000000)


In [6]:
#for bp in world.get_blueprint_library().filter('vehicle'):
    #print(bp.id)

In [7]:
# Set up the simulator in synchronous mode
settings = world.get_settings()
settings.synchronous_mode = True # Enables synchronous mode
settings.fixed_delta_seconds = 0.05
world.apply_settings(settings)

10842

In [8]:
# Get the blueprint for the vehicle you want
vehicle_bp = bp_lib.find('vehicle.nissan.patrol_2021') 

# Try spawning the vehicle at a randomly chosen spawn point
vehicle = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))
spectator = world.get_spectator()

In [9]:
# Move the spectator behind the vehicle 
#spectator = world.get_spectator() 
#transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-4,z=2.5)),vehicle.get_transform().rotation) 
#spectator.set_transform(transform) 

In [10]:
# Spawn vehicles
for i in range(50): 
    vehicle_bp = random.choice(bp_lib.filter('vehicle')) 
    npc = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points)) 

In [11]:
# Set the all vehicles in motion using the Traffic Manager
#for v in world.get_actors().filter('*vehicle*'): 
#    v.set_autopilot(True) 

In [12]:
# Spawn an RGB cammera with an offset from the vehicle center
#camera_bp = bp_lib.find('sensor.camera.rgb') 
#camera_init_trans = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-4,z=2.5)),vehicle.get_transform().rotation) 
#camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)

In [13]:
# camera on traffic lights
'''
traffic_light = world.get_actors().filter('traffic.traffic_light')[3]
print(traffic_light)  
p0 = traffic_light.get_transform()
camera_traffic = bp_lib.find('sensor.camera.rgb')
# Modify the attributes of the blueprint to set image resolution and field of view.
#camera_traffic.set_attribute('image_size_x', '1920')
#camera_traffic.set_attribute('image_size_y', '1080')
camera_traffic.set_attribute('fov', '110')
# Set the time in seconds between sensor captures
camera_traffic.set_attribute('sensor_tick', '2.0')
camera = world.spawn_actor(camera_traffic, p0, attach_to=traffic_light)
'''

"\ntraffic_light = world.get_actors().filter('traffic.traffic_light')[3]\nprint(traffic_light)  \np0 = traffic_light.get_transform()\ncamera_traffic = bp_lib.find('sensor.camera.rgb')\n# Modify the attributes of the blueprint to set image resolution and field of view.\n#camera_traffic.set_attribute('image_size_x', '1920')\n#camera_traffic.set_attribute('image_size_y', '1080')\ncamera_traffic.set_attribute('fov', '110')\n# Set the time in seconds between sensor captures\ncamera_traffic.set_attribute('sensor_tick', '2.0')\ncamera = world.spawn_actor(camera_traffic, p0, attach_to=traffic_light)\n"

In [31]:
#camera on spectator (drone)
#drone = world.get_spectator()
#transform = drone.get_transform()
#location = transform.location
#rotation = transform.rotation
location = carla.Location(x=14.0, y=29.0, z=6.0)
rotation = carla.Rotation(pitch=0.0, yaw=math.pi/2, roll=0.0)
transform = carla.Transform(location, rotation)
print(location, rotation)
# Set the spectator with an empty transform
#drone.set_transform(carla.Transform())

camera_drone = bp_lib.find('sensor.camera.rgb')
camera_drone.set_attribute('image_size_x', '1920')
camera_drone.set_attribute('image_size_y', '1080')
camera_drone.set_attribute('sensor_tick', '1.0')
camera_drone.set_attribute('fov', '110')
camera = world.spawn_actor(camera_drone, transform)


Location(x=14.000000, y=29.000000, z=6.000000) Rotation(pitch=0.000000, yaw=1.570796, roll=0.000000)


In [32]:
#Instance segmentation camera on drone
# drone = world.get_spectator()
# transform = drone.get_transform()
# location = transform.location
# rotation = transform.rotation
# # Set the spectator with an empty transform
# drone.set_transform(carla.Transform())

seg_camera = bp_lib.find('sensor.camera.instance_segmentation')
seg_camera.set_attribute('image_size_x', '1920')
seg_camera.set_attribute('image_size_y', '1080')
seg_camera.set_attribute('sensor_tick', '1.0')
seg_camera.set_attribute('fov', '110')
segmentation_camera = world.spawn_actor(seg_camera, transform)

In [33]:
# #semantic lidar on drone
# drone = world.get_spectator()
# transform = drone.get_transform()
# location = transform.location
# rotation = transform.rotation
# # Set the spectator with an empty transform
# drone.set_transform(carla.Transform())

sem_lidar = bp_lib.find('sensor.lidar.ray_cast_semantic')
sem_lidar.set_attribute('sensor_tick', '1.0')
lidar = world.spawn_actor(sem_lidar, transform)

In [34]:
#print lidar data
def semantic_lidar_data(point_cloud_data):
  for detection in point_cloud_data:
    print(detection)

In [35]:
# Start the camera saving data to disk (rgb)
#camera.listen(lambda image: image.save_to_disk('dataset/stationary_camera/rgb/%06d.png' % image.frame))

In [36]:
# Save segmented images
#segmentation_camera.listen(lambda image: image.save_to_disk('dataset/stationary_camera/seg_cam/%06d.png' % image.frame))

In [37]:
#Save Lidar info
#lidar.listen(lambda point_cloud_data: semantic_lidar_data(point_cloud_data))

In [38]:
#camera.stop()
#segmentation_camera.stop()
#lidar.stop()

In [39]:
#Camera geometric projection -> 3D points
def build_projection_matrix(w, h, fov, is_behind_camera=False):
    focal = w / (2.0 * np.tan(fov * np.pi / 360.0))
    K = np.identity(3)

    if is_behind_camera:
        K[0, 0] = K[1, 1] = -focal
    else:
        K[0, 0] = K[1, 1] = focal

    K[0, 2] = w / 2.0
    K[1, 2] = h / 2.0
    return K

In [40]:
#3D to 2D
def get_image_point(loc, K, w2c):
        # Calculate 2D projection of 3D coordinate

        # Format the input coordinate (loc is a carla.Position object)
        point = np.array([loc.x, loc.y, loc.z, 1])
        # transform to camera coordinates
        point_camera = np.dot(w2c, point)

        # New we must change from UE4's coordinate system to an "standard"
        # (x, y ,z) -> (y, -z, x)
        # and we remove the fourth componebonent also
        point_camera = [point_camera[1], -point_camera[2], point_camera[0]]

        # now project 3D->2D using the camera matrix
        point_img = np.dot(K, point_camera)
        # normalize
        point_img[0] /= point_img[2]
        point_img[1] /= point_img[2]

        return point_img[0:2]

In [41]:
# Get the world to camera matrix
world_2_camera = np.array(camera.get_transform().get_inverse_matrix())

# Get the attributes from the camera
image_w = camera_drone.get_attribute("image_size_x").as_int()
image_h = camera_drone.get_attribute("image_size_y").as_int()
fov = camera_drone.get_attribute("fov").as_float()

# Calculate the camera projection matrix to project from 3D -> 2D
K = build_projection_matrix(image_w, image_h, fov)
K_b = build_projection_matrix(image_w, image_h, fov, is_behind_camera=True)

In [42]:
image_queue = queue.Queue()
camera.listen(image_queue.put)

In [48]:
width = 1920
height = 1080
seg_camera.set_attribute('image_size_y', '1080')

pygame.init()
window = pygame.display.set_mode(size=(width,height))
font = pygame.font.SysFont(None, 24)
video_surf = None

world.tick()
# Set the all vehicles in motion using the Traffic Manager
for v in world.get_actors().filter('*vehicle*'): 
    v.set_autopilot(True)

'''
simulation_dataset = {

    "images": [...,
        {
        "file_name": "023235.png",
        "height": 600,
        "width": 800,
        "id": 23235
    },
    ],

    "annotations": [
        ...,
        {
            "image_id": 23235,
            "bbox": [503.3, 310.4, 118.3, 78.3]
        },
        ...
    ]
}
'''
simulation_dataset = {}
simulation_dataset["images"] = []
simulation_dataset["annotations"] = []

#render 2D bboxes
running = True
tick = 0
max_tick = 12000
while running:
    tick = tick + 1
    print(tick)
    if (tick > max_tick):
        break
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
    # Retrieve and reshape the image
    world.tick()
    image = None
    try:
        image = image_queue.get(timeout=1)
        print("Received image!")
    except:
        print("No image!")
        continue

    img = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    img_filename = 'dataset/stationary_camera/rgb/%06d.png' % image.frame
    img_bgr = img[:,:, :3]
    #print(img_bg)
    img_rgb = img_bgr[:,:, ::-1]
    img_PIL = Image.fromarray(img_rgb)
    img_PIL.save(img_filename)
    simulation_dataset["images"].append({
        "file_name": img_filename,
        "height": 1920,
        "width": 1080,
        "id": image.frame
    })

    # Get the camera matrix 
    world_2_camera = np.array(camera.get_transform().get_inverse_matrix())

    for npc in world.get_actors().filter('*vehicle*'):

        # Filter out the ego vehicle
        if npc.id != vehicle.id:

            bb = npc.bounding_box
            dist = npc.get_transform().location.distance(transform.location)

            # Filter for the vehicles within 50m
            if dist < 70:

            # Calculate the dot product between the forward vector
            # of the vehicle and the vector between the vehicle
            # and the other vehicle. We threshold this dot product
            # to limit to drawing bounding boxes IN FRONT OF THE CAMERA
                forward_vec = transform.get_forward_vector()
                ray = npc.get_transform().location - transform.location

                if forward_vec.dot(ray) > 0:
                    p1 = get_image_point(bb.location, K, world_2_camera)
                    verts = [v for v in bb.get_world_vertices(npc.get_transform())]
                    x_max = -10000
                    x_min = 10000
                    y_max = -10000
                    y_min = 10000

                    for vert in verts:
                        p = get_image_point(vert, K, world_2_camera)
                        # Find the rightmost vertex
                        if p[0] > x_max:
                            x_max = p[0]
                        # Find the leftmost vertex
                        if p[0] < x_min:
                            x_min = p[0]
                        # Find the highest vertex
                        if p[1] > y_max:
                            y_max = p[1]
                        # Find the lowest  vertex
                        if p[1] < y_min:
                            y_min = p[1]

                    cv2.line(img, (int(x_min),int(y_min)), (int(x_max),int(y_min)), (0,0,255, 255), 1)
                    cv2.line(img, (int(x_min),int(y_max)), (int(x_max),int(y_max)), (0,0,255, 255), 1)
                    cv2.line(img, (int(x_min),int(y_min)), (int(x_min),int(y_max)), (0,0,255, 255), 1)
                    cv2.line(img, (int(x_max),int(y_min)), (int(x_max),int(y_max)), (0,0,255, 255), 1)
                    bbox = [x_min, y_min, x_max, y_max]
                    rounded_bbox = [round(value) for value in bbox]
                    print(rounded_bbox)
                    simulation_dataset["annotations"].append({
                        "image_id": image.frame,
                        "bbox": rounded_bbox
                    })

    video_surf = pygame.image.frombuffer(img, (image.width, image.height), "BGRA")
    window.blit(video_surf, (0,0))
    pygame.display.flip()
    #pygame.time.wait(5)
pygame.quit()

with open('simulation_data.json', 'w') as json_file:
    json.dump(simulation_dataset, json_file)
    


1
No image!
2


In [44]:
pygame.quit()

In [45]:
# box = vehicle.bounding_box
# print(box.location)         # Location relative to the vehicle.
# print(box.extent)
#cv2.destroyAllWindows()

Location(x=0.028380, y=0.000002, z=1.016716)
Vector3D(x=2.782914, y=1.074983, z=1.022574)
